Gensim models available
https://github.com/piskvorky/gensim-data

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 71.1 MB/s eta 0:00:00


In [2]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
wv.similarity(w1="better", w2="good")

np.float32(0.6120729)

In [4]:
wv.most_similar("good")

[('great', 0.7291510105133057),
 ('bad', 0.7190051078796387),
 ('terrific', 0.6889115571975708),
 ('decent', 0.6837348341941833),
 ('nice', 0.6836092472076416),
 ('excellent', 0.644292950630188),
 ('fantastic', 0.6407778263092041),
 ('better', 0.6120728850364685),
 ('solid', 0.5806034803390503),
 ('lousy', 0.576420247554779)]

In [ ]:
vector = wv['good']
vector.shape

(300,)

In [8]:
import pandas as pd
import numpy as np


In [9]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df['sentiment']= df['sentiment'].replace({'positive':1,'negative':0})
df.head()

/tmp/ipython-input-4145410600.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment']= df['sentiment'].replace({'positive':1,'negative':0})


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
df.isnull().sum()

,0
review,0
sentiment,0


In [12]:
!python -m spacy download en_core_web_lg

import spacy
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
def preprocess(text):
  doc = nlp(text)
  filtered_token = []
  for token in doc:
    if token.is_stop and token.is_punct and token.is_digit:
      continue
    else:
      filtered_token.append(token.lemma_)

  return wv.get_mean_vector(filtered_token)

In [17]:
df['vector'] = df['review'].apply(lambda x: preprocess(x))

In [18]:
df.head()

,review,sentiment,vector
0,One of the other reviewers has mentioned that ...,1,"[0.01429554, 0.006870111, 0.021525184, 0.04915..."
1,A wonderful little production. <br /><br />The...,1,"[0.024627518, 0.014157958, 0.0040722913, 0.042..."
2,I thought this was a wonderful way to spend ti...,1,"[0.015319369, 0.012464422, 0.013687313, 0.0523..."
3,Basically there's a family where a little boy ...,0,"[0.018154278, 0.007264626, 0.006620961, 0.0550..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[0.022093147, 0.010015907, 0.015895324, 0.0415..."


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.sentiment.values, test_size=0.2, random_state=42)

In [24]:
X_train.shape

(40000,)

In [25]:
# Convert the Series of arrays into a 2D NumPy array
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [26]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50,criterion='entropy')

model.fit(X_train_2d, y_train)

from sklearn.metrics import classification_report

y_pred = model.predict(X_test_2d)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.80      4961
           1       0.80      0.79      0.80      5039

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



In [27]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5,metric='euclidean')

model.fit(X_train_2d, y_train)

from sklearn.metrics import classification_report

y_pred = model.predict(X_test_2d)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.73      0.80      0.76      4961
           1       0.78      0.70      0.74      5039

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81      4961
           1       0.82      0.81      0.81      5039

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000

